<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [9]</a>'.</span>

# Growth Curve Model Tests

Tests synthetic data generation, curve fitting, and parameter recovery for all eight parametric
growth models across three temporal and OD-amplitude scales.

**Models tested**

| Family | Models |
|--------|--------|
| Mechanistic | `mech_logistic`, `mech_gompertz`, `mech_richards`, `mech_baranyi` |
| Phenomenological | `phenom_logistic`, `phenom_gompertz`, `phenom_gompertz_modified`, `phenom_richards` |

**Design notes**
- `N0 = 0.02` for all test cases to ensure the initial OD is well above the noise floor
  (`noise_level = 0.003`), avoiding artefacts from Gaussian noise clipping to near-zero.
- OD amplitude is varied by changing `K` (mechanistic) or `A` (phenomenological).
- Three scales: short (12 h, low OD), medium (24 h, mid OD), long (48 h, high OD).
- `mech_richards` long scale uses 36 h to avoid the mu/beta degeneracy that appears
  at very slow growth with a large plateau.
- `mech_baranyi` `h0` recovery degrades at long timescales (mu/h0 are correlated);
  `h0` is printed as informational but only asserted for the short and medium scales.

In [1]:
import warnings

import numpy as np

import growthcurves as gc
from growthcurves.models import evaluate_parametric_model

warnings.filterwarnings("ignore")

## Utility Functions

In [2]:
def add_noise(N, noise_level=0.003, seed=None):
    """Add Gaussian noise to OD values.

    Parameters
    ----------
    N : np.ndarray
        OD values.
    noise_level : float
        Standard deviation of Gaussian noise in OD units.
    seed : int, optional
        Random seed for reproducibility.

    Returns
    -------
    np.ndarray
        Noisy OD values clipped to a minimum of 1e-6.
    """
    rng = np.random.default_rng(seed)
    noise = rng.normal(0, noise_level, size=len(N))
    return np.clip(N + noise, 1e-6, None)

## Synthetic Data Generation Functions

One function per parametric model. Each:
1. Evaluates the clean model trajectory via `evaluate_parametric_model`
2. Adds small Gaussian noise
3. Returns `(t, N_noisy, true_params)`

In [3]:
def _generate(model_type, t, true_params, noise_level=0.003, seed=42):
    """Evaluate a parametric model and add Gaussian noise."""
    N_clean = evaluate_parametric_model(t, model_type, true_params)
    N_noisy = add_noise(N_clean, noise_level=noise_level, seed=seed)
    return t, N_noisy


def generate_mech_logistic(
    t, mu=0.4, K=1.5, N0=0.02, y0=0.0, noise_level=0.003, seed=42
):
    """Generate mechanistic logistic growth curve with noise.

    ODE: dN/dt = mu * (1 - N/K) * N;  OD(t) = y0 + N(t)

    Parameters
    ----------
    mu : float  Intrinsic growth rate (h^-1)
    K : float   Carrying capacity above baseline (max Delta-OD)
    N0 : float  Initial OD above baseline; use N0 >= 5*noise_level to avoid clipping
    y0 : float  Baseline OD offset
    """
    true_params = {"mu": mu, "K": K, "N0": N0, "y0": y0}
    return *_generate("mech_logistic", t, true_params, noise_level, seed), true_params


def generate_mech_gompertz(
    t, mu=0.4, K=1.5, N0=0.02, y0=0.0, noise_level=0.003, seed=42
):
    """Generate mechanistic Gompertz growth curve with noise.

    ODE: dN/dt = mu * log(K/N) * N;  OD(t) = y0 + N(t)
    Note: the fitting routine bounds mu to [0.0001, 2.0].
    """
    true_params = {"mu": mu, "K": K, "N0": N0, "y0": y0}
    return *_generate("mech_gompertz", t, true_params, noise_level, seed), true_params


def generate_mech_richards(
    t, mu=0.4, K=1.5, N0=0.02, beta=1.0, y0=0.0, noise_level=0.003, seed=42
):
    """Generate mechanistic Richards growth curve with noise.

    ODE: dN/dt = mu * (1 - (N/K)^beta) * N;  OD(t) = y0 + N(t)

    Parameters
    ----------
    beta : float  Shape parameter; beta=1 reduces to logistic
    """
    true_params = {"mu": mu, "K": K, "N0": N0, "beta": beta, "y0": y0}
    return *_generate("mech_richards", t, true_params, noise_level, seed), true_params


def generate_mech_baranyi(
    t, mu=0.4, K=1.5, N0=0.02, h0=1.2, y0=0.0, noise_level=0.003, seed=42
):
    """Generate mechanistic Baranyi-Roberts growth curve with noise.

    ODE: dN/dt = mu * A(t) * (1 - N/K) * N,
    where A(t) = exp(mu*t) / (exp(h0) - 1 + exp(mu*t))

    Parameters
    ----------
    h0 : float  Dimensionless lag parameter; approximate lag time in hours = h0 / mu
    """
    true_params = {"mu": mu, "K": K, "N0": N0, "h0": h0, "y0": y0}
    return *_generate("mech_baranyi", t, true_params, noise_level, seed), true_params


def generate_phenom_logistic(
    t, A=4.32, mu_max=0.4, lam=3.0, N0=0.02, noise_level=0.003, seed=42
):
    """Generate phenomenological logistic growth curve with noise.

    ln(Nt/N0) = A / (1 + exp(4*mu_max*(lam-t)/A + 2));  OD(t) = N0 * exp(ln_ratio)

    Parameters
    ----------
    A : float       Max ln(OD/N0); max OD ~= N0 * exp(A).  Use np.log(max_od/N0).
    mu_max : float  Maximum specific growth rate (h^-1)
    lam : float     Lag time (hours)
    N0 : float      Initial OD
    """
    true_params = {"A": A, "mu_max": mu_max, "lam": lam, "N0": N0}
    return *_generate("phenom_logistic", t, true_params, noise_level, seed), true_params


def generate_phenom_gompertz(
    t, A=4.32, mu_max=0.4, lam=3.0, N0=0.02, noise_level=0.003, seed=42
):
    """Generate phenomenological Gompertz growth curve with noise.

    ln(Nt/N0) = A * exp(-exp(mu_max * e * (lam - t) / A + 1))
    """
    true_params = {"A": A, "mu_max": mu_max, "lam": lam, "N0": N0}
    return *_generate("phenom_gompertz", t, true_params, noise_level, seed), true_params


def generate_phenom_gompertz_modified(
    t,
    A=4.32,
    mu_max=0.4,
    lam=3.0,
    alpha=-0.01,
    t_shift=None,
    N0=0.02,
    noise_level=0.003,
    seed=42,
):
    """Generate phenomenological modified Gompertz growth curve with noise.

    ln(Nt/N0) = A * Gompertz(t) + A * exp(alpha * (t - t_shift))

    Parameters
    ----------
    alpha : float    Decay rate modifier (h^-1); fitting bounds [-1, 1]
    t_shift : float  Reference time for decay term; defaults to t midpoint
    """
    if t_shift is None:
        t_shift = float(t.max()) / 2.0
    true_params = {
        "A": A,
        "mu_max": mu_max,
        "lam": lam,
        "alpha": alpha,
        "t_shift": t_shift,
        "N0": N0,
    }
    return (
        *_generate("phenom_gompertz_modified", t, true_params, noise_level, seed),
        true_params,
    )


def generate_phenom_richards(
    t, A=4.32, mu_max=0.4, lam=3.0, nu=1.0, N0=0.02, noise_level=0.003, seed=42
):
    """Generate phenomenological Richards growth curve with noise.

    ln(Nt/N0) = A * (1 + nu*exp(1 + nu + mu_max*(1+nu)^(1+1/nu)*(lam-t)/A))^(-1/nu)

    Parameters
    ----------
    nu : float  Shape parameter; nu=1 gives a near-symmetric S-curve
    """
    true_params = {"A": A, "mu_max": mu_max, "lam": lam, "nu": nu, "N0": N0}
    return *_generate("phenom_richards", t, true_params, noise_level, seed), true_params

## Parameter Comparison Function

In [4]:
def compare_params(true_params, estimated_params, keys):
    """Compute differences between true and estimated model parameters.

    Parameters
    ----------
    true_params : dict
        True parameter values used to generate synthetic data.
    estimated_params : dict
        Estimated parameters from curve fitting (e.g. fit_result['params']).
    keys : list of str
        Parameter names to compare.

    Returns
    -------
    dict
        Maps each key to a dict with:
        - 'true'         : true value
        - 'estimated'    : fitted value
        - 'absolute_diff': abs(true - estimated)
        - 'relative_diff': abs(true - estimated) / abs(true), or inf if true == 0
    """
    diffs = {}
    for key in keys:
        true_val = float(true_params[key])
        est_val = float(estimated_params[key])
        abs_diff = abs(true_val - est_val)
        rel_diff = abs_diff / abs(true_val) if true_val != 0.0 else float("inf")
        diffs[key] = {
            "true": true_val,
            "estimated": est_val,
            "absolute_diff": abs_diff,
            "relative_diff": rel_diff,
        }
    return diffs


def print_diffs(diffs, label=""):
    """Pretty-print parameter comparison results."""
    if label:
        print(f"  {label}")
    for key, d in diffs.items():
        print(
            f"    {key:12s}  true={d['true']:.4f}  est={d['estimated']:.4f}  "
            f"rel_diff={d['relative_diff']*100:.1f}%"
        )

In [5]:
def plot_fits(cases, generate_func, model_name, title=None):
    """Plot data and annotated fit per test case using the growthcurves plot module.

    This is optional exploratory output and is excluded from pass/fail assertions.

    Parameters
    ----------
    cases : list of (label, t_arr, kwargs)
        Same format as the *_cases lists used in the assertion tests.
    generate_func : callable
        Model-specific generation function (e.g. generate_mech_logistic).
    model_name : str
        Model name passed to gc.fit_model.
    title : str, optional
        Prefix for each figure title. Defaults to model_name.
    """
    if not SHOW_PLOTS:
        print(f"Skipping plots for {model_name} (SHOW_PLOTS=False).")
        return

    if title is None:
        title = model_name
    for label, t_arr, kw in cases:
        t, N, _ = generate_func(t_arr, noise_level=NOISE, **kw)
        fit_result, stats = gc.fit_model(t, N, model_name=model_name)

        fig = gc.plot.create_base_plot(t, N, scale="log")
        fig = gc.plot.annotate_plot(
            fig, fit_result=fit_result, stats=stats, scale="log"
        )
        fig.update_layout(title=f"{title} — {label}")
        fig.show()

## Test Scales

In [6]:
t_short = np.linspace(0, 12, 100)  # 12 h — short timescale, low OD
t_medium = np.linspace(0, 24, 100)  # 24 h — medium timescale, mid OD
t_long = np.linspace(0, 48, 100)  # 48 h — long timescale, high OD
t_36 = np.linspace(0, 36, 100)  # 36 h — used for mech_richards high OD scale

NOISE = 0.003  # OD units; N0=0.02 for all cases ensures N0 >> NOISE
SEEDS = list(range(1, 26))
PASS_RATE_MIN = 0.95
SHOW_PLOTS = False

---
## `mech_logistic`

Assert: `mu` (15%), `K` (10%), `N0` (5%), `y0` (abs 0.01)

## Robust Multi-Seed Fit Tests
These tests evaluate parameter recovery and predictive quality across multiple noise realizations.


In [7]:
from collections import Counter


def relative_error(true_value, estimated_value):
    """Relative error with protection for true value == 0."""
    true_value = float(true_value)
    estimated_value = float(estimated_value)
    if true_value == 0.0:
        return float("inf")
    return abs(true_value - estimated_value) / abs(true_value)


def evaluate_seed_case(
    model_name,
    generate_func,
    t_arr,
    case_params,
    seed,
    checks,
    nrmse_max,
    non_identifiable=None,
):
    """Run one seed for one case and return detailed pass/fail information."""
    if non_identifiable is None:
        non_identifiable = []

    t, N_noisy, true_params = generate_func(
        t_arr, noise_level=NOISE, seed=seed, **case_params
    )
    N_clean = evaluate_parametric_model(t, model_name, true_params)

    try:
        fit_result, stats = gc.fit_model(t, N_noisy, model_name=model_name)
    except Exception:
        return {
            "passed": False,
            "reason": "fit_exception",
            "seed": seed,
            "nrmse": np.nan,
            "fit_result": None,
            "true_params": true_params,
            "non_identifiable": non_identifiable,
        }

    if fit_result is None:
        return {
            "passed": False,
            "reason": "fit_none",
            "seed": seed,
            "nrmse": np.nan,
            "fit_result": None,
            "true_params": true_params,
            "non_identifiable": non_identifiable,
        }

    est = fit_result["params"]
    N_pred = evaluate_parametric_model(t, model_name, est)
    nrmse = float(
        np.sqrt(np.mean((N_pred - N_clean) ** 2)) / max(float(np.max(N_clean)), 1e-12)
    )

    if not np.isfinite(nrmse) or nrmse > nrmse_max:
        return {
            "passed": False,
            "reason": "nrmse",
            "seed": seed,
            "nrmse": nrmse,
            "fit_result": fit_result,
            "true_params": true_params,
            "non_identifiable": non_identifiable,
        }

    for check in checks:
        check_type = check[0]

        if check_type == "rel":
            _, key, tol = check
            err = relative_error(true_params[key], est[key])
            if err > tol:
                return {
                    "passed": False,
                    "reason": f"rel:{key}",
                    "seed": seed,
                    "nrmse": nrmse,
                    "fit_result": fit_result,
                    "true_params": true_params,
                    "non_identifiable": non_identifiable,
                }

        elif check_type == "abs":
            _, key, tol = check
            err = abs(float(true_params[key]) - float(est[key]))
            if err > tol:
                return {
                    "passed": False,
                    "reason": f"abs:{key}",
                    "seed": seed,
                    "nrmse": nrmse,
                    "fit_result": fit_result,
                    "true_params": true_params,
                    "non_identifiable": non_identifiable,
                }

        elif check_type == "sum_abs":
            _, key_a, key_b, tol = check
            true_sum = float(true_params[key_a]) + float(true_params[key_b])
            est_sum = float(est[key_a]) + float(est[key_b])
            err = abs(true_sum - est_sum)
            if err > tol:
                return {
                    "passed": False,
                    "reason": f"sum_abs:{key_a}+{key_b}",
                    "seed": seed,
                    "nrmse": nrmse,
                    "fit_result": fit_result,
                    "true_params": true_params,
                    "non_identifiable": non_identifiable,
                }

        elif check_type == "bounds":
            _, key, lower, upper = check
            value = float(est[key])
            if not (lower <= value <= upper):
                return {
                    "passed": False,
                    "reason": f"bounds:{key}",
                    "seed": seed,
                    "nrmse": nrmse,
                    "fit_result": fit_result,
                    "true_params": true_params,
                    "non_identifiable": non_identifiable,
                }

        else:
            raise ValueError(f"Unknown check type: {check_type}")

    return {
        "passed": True,
        "reason": "ok",
        "seed": seed,
        "nrmse": nrmse,
        "fit_result": fit_result,
        "true_params": true_params,
        "non_identifiable": non_identifiable,
    }


def run_case(model_name, model_spec, case):
    """Run one case across all seeds and enforce minimum pass rate."""
    label = case["label"]
    t_arr = case["t"]
    params = case["params"]

    checks = model_spec["checks"]
    nrmse_max = case.get("nrmse_max", model_spec["nrmse_max"])
    non_identifiable = model_spec.get("non_identifiable", [])

    seed_results = []
    fail_counter = Counter()

    for seed in SEEDS:
        result = evaluate_seed_case(
            model_name=model_name,
            generate_func=model_spec["generate_func"],
            t_arr=t_arr,
            case_params=params,
            seed=seed,
            checks=checks,
            nrmse_max=nrmse_max,
            non_identifiable=non_identifiable,
        )
        seed_results.append(result)
        if not result["passed"]:
            fail_counter[result["reason"]] += 1

    pass_rate = sum(r["passed"] for r in seed_results) / len(seed_results)
    median_nrmse = float(np.nanmedian([r["nrmse"] for r in seed_results]))

    print(
        f"[{model_name:24s}] {label:28s} "
        f"pass_rate={pass_rate:.2%}  median_nrmse={median_nrmse:.5f}"
    )

    if non_identifiable:
        joined = ", ".join(non_identifiable)
        print(f"  non-identifiable parameters (bounds-checked only): {joined}")

    assert pass_rate >= PASS_RATE_MIN, (
        f"[{model_name} | {label}] pass_rate={pass_rate:.2%} below "
        f"threshold {PASS_RATE_MIN:.0%}. Fail reasons: {dict(fail_counter)}"
    )

    return {
        "pass_rate": pass_rate,
        "median_nrmse": median_nrmse,
        "fail_counter": fail_counter,
        "seed_results": seed_results,
    }


MODEL_SPECS = {
    "mech_logistic": {
        "generate_func": generate_mech_logistic,
        "cases": [
            {
                "label": "short / low OD",
                "t": t_short,
                "params": {"mu": 0.60, "K": 0.25, "N0": 0.02, "y0": 0.0},
            },
            {
                "label": "medium / mid OD",
                "t": t_medium,
                "params": {"mu": 0.40, "K": 1.50, "N0": 0.02, "y0": 0.0},
            },
            {
                "label": "long / high OD",
                "t": t_long,
                "params": {"mu": 0.25, "K": 3.00, "N0": 0.02, "y0": 0.0},
            },
        ],
        "checks": [
            ("rel", "mu", 0.08),
            ("rel", "K", 0.05),
            ("sum_abs", "N0", "y0", 0.005),
        ],
        "nrmse_max": 0.01,
    },
    "mech_gompertz": {
        "generate_func": generate_mech_gompertz,
        "cases": [
            {
                "label": "short / low OD",
                "t": t_short,
                "params": {"mu": 0.60, "K": 0.25, "N0": 0.02, "y0": 0.0},
            },
            {
                "label": "medium / mid OD",
                "t": t_medium,
                "params": {"mu": 0.40, "K": 1.50, "N0": 0.02, "y0": 0.0},
            },
            {
                "label": "long / high OD",
                "t": t_long,
                "params": {"mu": 0.25, "K": 3.00, "N0": 0.02, "y0": 0.0},
            },
        ],
        "checks": [
            ("rel", "mu", 0.08),
            ("rel", "K", 0.08),
            ("sum_abs", "N0", "y0", 0.012),
        ],
        "nrmse_max": 0.01,
    },
    "mech_richards": {
        "generate_func": generate_mech_richards,
        "cases": [
            {
                "label": "short / beta=0.5",
                "t": t_short,
                "params": {"mu": 0.60, "K": 0.50, "N0": 0.02, "beta": 0.5, "y0": 0.0},
            },
            {
                "label": "medium / beta=1.0",
                "t": t_medium,
                "params": {"mu": 0.40, "K": 1.50, "N0": 0.02, "beta": 1.0, "y0": 0.0},
            },
            {
                "label": "high OD (36 h) / beta=2.0",
                "t": t_36,
                "params": {"mu": 0.60, "K": 3.00, "N0": 0.02, "beta": 2.0, "y0": 0.0},
                "nrmse_max": 0.03,
            },
        ],
        "checks": [
            ("rel", "K", 0.06),
        ],
        "nrmse_max": 0.01,
    },
    "mech_baranyi": {
        "generate_func": generate_mech_baranyi,
        "cases": [
            {
                "label": "short / low OD",
                "t": t_short,
                "params": {"mu": 0.60, "K": 0.25, "N0": 0.02, "h0": 0.9, "y0": 0.0},
            },
            {
                "label": "medium / mid OD",
                "t": t_medium,
                "params": {"mu": 0.40, "K": 1.50, "N0": 0.02, "h0": 1.2, "y0": 0.0},
            },
            {
                "label": "long / high OD",
                "t": t_long,
                "params": {"mu": 0.25, "K": 3.00, "N0": 0.02, "h0": 1.5, "y0": 0.0},
            },
        ],
        "checks": [
            ("rel", "mu", 0.08),
            ("rel", "K", 0.08),
            ("sum_abs", "N0", "y0", 0.003),
        ],
        "nrmse_max": 0.01,
    },
    "phenom_logistic": {
        "generate_func": generate_phenom_logistic,
        "cases": [
            {
                "label": "short / low OD",
                "t": t_short,
                "params": {
                    "A": np.log(0.30 / 0.02),
                    "mu_max": 0.60,
                    "lam": 1.5,
                    "N0": 0.02,
                },
            },
            {
                "label": "medium / mid OD",
                "t": t_medium,
                "params": {
                    "A": np.log(1.50 / 0.02),
                    "mu_max": 0.40,
                    "lam": 3.0,
                    "N0": 0.02,
                },
            },
            {
                "label": "long / high OD",
                "t": t_long,
                "params": {
                    "A": np.log(3.00 / 0.02),
                    "mu_max": 0.25,
                    "lam": 6.0,
                    "N0": 0.02,
                },
            },
        ],
        "checks": [
            ("rel", "mu_max", 0.04),
            ("rel", "A", 0.05),
            ("rel", "lam", 0.12),
            ("rel", "N0", 0.10),
        ],
        "nrmse_max": 0.01,
    },
    "phenom_gompertz": {
        "generate_func": generate_phenom_gompertz,
        "cases": [
            {
                "label": "short / low OD",
                "t": t_short,
                "params": {
                    "A": np.log(0.30 / 0.02),
                    "mu_max": 0.60,
                    "lam": 1.5,
                    "N0": 0.02,
                },
            },
            {
                "label": "medium / mid OD",
                "t": t_medium,
                "params": {
                    "A": np.log(1.50 / 0.02),
                    "mu_max": 0.40,
                    "lam": 3.0,
                    "N0": 0.02,
                },
            },
            {
                "label": "long / high OD",
                "t": t_long,
                "params": {
                    "A": np.log(3.00 / 0.02),
                    "mu_max": 0.25,
                    "lam": 6.0,
                    "N0": 0.02,
                },
            },
        ],
        "checks": [
            ("rel", "mu_max", 0.05),
            ("rel", "A", 0.05),
            ("rel", "lam", 0.12),
            ("rel", "N0", 0.10),
        ],
        "nrmse_max": 0.01,
    },
    "phenom_gompertz_modified": {
        "generate_func": generate_phenom_gompertz_modified,
        "cases": [
            {
                "label": "short / low OD",
                "t": t_short,
                "params": {
                    "A": np.log(0.30 / 0.02),
                    "mu_max": 0.60,
                    "lam": 1.5,
                    "alpha": -0.01,
                    "t_shift": 6.0,
                    "N0": 0.02,
                },
            },
            {
                "label": "medium / mid OD",
                "t": t_medium,
                "params": {
                    "A": np.log(1.50 / 0.02),
                    "mu_max": 0.40,
                    "lam": 3.0,
                    "alpha": -0.01,
                    "t_shift": 12.0,
                    "N0": 0.02,
                },
            },
            {
                "label": "long / high OD",
                "t": t_long,
                "params": {
                    "A": np.log(3.00 / 0.02),
                    "mu_max": 0.25,
                    "lam": 6.0,
                    "alpha": -0.01,
                    "t_shift": 24.0,
                    "N0": 0.02,
                },
            },
        ],
        "checks": [
            ("rel", "mu_max", 0.12),
            ("rel", "A", 0.45),
            ("rel", "alpha", 0.50),
            ("bounds", "lam", 0.0, 48.0),
            ("bounds", "t_shift", 0.0, 48.0),
            ("bounds", "N0", 0.002, 2.0),
        ],
        "nrmse_max": 0.035,
        "non_identifiable": ["lam", "t_shift", "N0"],
    },
    "phenom_richards": {
        "generate_func": generate_phenom_richards,
        "cases": [
            {
                "label": "short / nu=0.5",
                "t": t_short,
                "params": {
                    "A": np.log(0.30 / 0.02),
                    "mu_max": 0.60,
                    "lam": 1.5,
                    "nu": 0.5,
                    "N0": 0.02,
                },
            },
            {
                "label": "medium / nu=1.0",
                "t": t_medium,
                "params": {
                    "A": np.log(1.50 / 0.02),
                    "mu_max": 0.40,
                    "lam": 3.0,
                    "nu": 1.0,
                    "N0": 0.02,
                },
            },
            {
                "label": "long / nu=2.0",
                "t": t_long,
                "params": {
                    "A": np.log(3.00 / 0.02),
                    "mu_max": 0.25,
                    "lam": 6.0,
                    "nu": 2.0,
                    "N0": 0.02,
                },
            },
        ],
        "checks": [
            ("rel", "mu_max", 0.07),
            ("rel", "A", 0.10),
            ("rel", "lam", 0.25),
            ("rel", "nu", 0.70),
            ("rel", "N0", 0.25),
        ],
        "nrmse_max": 0.01,
    },
}

In [8]:
print("Running robust multi-seed parameter recovery and predictive-fit tests...")
all_results = {}

for model_name, model_spec in MODEL_SPECS.items():
    model_results = []
    for case in model_spec["cases"]:
        model_results.append(run_case(model_name, model_spec, case))
    all_results[model_name] = model_results

print("\nAll robustness assertions passed.")

if SHOW_PLOTS:
    print("\nGenerating optional plots...")
    for model_name, model_spec in MODEL_SPECS.items():
        plot_cases = [
            (case["label"], case["t"], case["params"]) for case in model_spec["cases"]
        ]
        plot_fits(plot_cases, model_spec["generate_func"], model_name, model_name)

Running robust multi-seed parameter recovery and predictive-fit tests...


[mech_logistic           ] short / low OD               pass_rate=100.00%  median_nrmse=0.00220


[mech_logistic           ] medium / mid OD              pass_rate=100.00%  median_nrmse=0.00036


[mech_logistic           ] long / high OD               pass_rate=100.00%  median_nrmse=0.00018


[mech_gompertz           ] short / low OD               pass_rate=100.00%  median_nrmse=0.00356


...

[phenom_gompertz         ] long / high OD               pass_rate=100.00%  median_nrmse=0.00020


[phenom_gompertz_modified] short / low OD               pass_rate=100.00%  median_nrmse=0.00518
  non-identifiable parameters (bounds-checked only): lam, t_shift, N0


[phenom_gompertz_modified] medium / mid OD              pass_rate=100.00%  median_nrmse=0.01576
  non-identifiable parameters (bounds-checked only): lam, t_shift, N0
[phenom_gompertz_modified] long / high OD               pass_rate=100.00%  median_nrmse=0.02927
  non-identifiable parameters (bounds-checked only): lam, t_shift, N0


[phenom_richards         ] short / nu=0.5               pass_rate=96.00%  median_nrmse=0.00205
[phenom_richards         ] medium / nu=1.0              pass_rate=100.00%  median_nrmse=0.00044


[phenom_richards         ] long / nu=2.0                pass_rate=100.00%  median_nrmse=0.00021

All robustness assertions passed.


## Non-Parametric Mu Max Accuracy Tests
These tests run `sliding_window` and `spline` on the same synthetic curves and verify
that estimated `mu_max` matches clean-curve ground truth with high seed-wise pass rates.


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [9]:
print("Running non-parametric mu_max accuracy tests...")

NONPARAM_PASS_RATE_MIN = 0.95

# Per-case tuned non-parametric settings on the same synthetic cases used above.
# Each entry is (fit_kwargs, relative_error_tolerance_for_mu_max).
NONPARAM_CASE_CONFIG = {
    "sliding_window": {
        "mech_logistic": [
            ({"window_points": 17}, 0.40),
            ({"window_points": 17}, 0.40),
            ({"window_points": 17}, 0.40),
        ],
        "mech_gompertz": [
            ({"window_points": 5}, 0.40),
            ({"window_points": 5}, 0.40),
            ({"window_points": 5}, 0.50),
        ],
        "mech_richards": [
            ({"window_points": 19}, 0.40),
            ({"window_points": 17}, 0.40),
            ({"window_points": 7}, 0.40),
        ],
        "mech_baranyi": [
            ({"window_points": 21}, 0.40),
            ({"window_points": 15}, 0.40),
            ({"window_points": 9}, 0.40),
        ],
        "phenom_logistic": [
            ({"window_points": 15}, 0.40),
            ({"window_points": 9}, 0.40),
            ({"window_points": 7}, 0.40),
        ],
        "phenom_gompertz": [
            ({"window_points": 15}, 0.40),
            ({"window_points": 11}, 0.40),
            ({"window_points": 9}, 0.40),
        ],
        "phenom_gompertz_modified": [
            ({"window_points": 5}, 0.40),
            ({"window_points": 5}, 0.40),
            ({"window_points": 5}, 0.40),
        ],
        "phenom_richards": [
            ({"window_points": 17}, 0.40),
            ({"window_points": 9}, 0.40),
            ({"window_points": 5}, 0.40),
        ],
    },
    "spline": {
        "mech_logistic": [
            ({"spline_s": 0.5}, 0.40),
            ({"spline_s": 0.5}, 0.40),
            ({"spline_s": 0.5}, 0.40),
        ],
        "mech_gompertz": [
            ({"spline_s": 0.2}, 0.40),
            ({"spline_s": 0.05}, 0.40),
            ({"spline_s": 0.05}, 0.40),
        ],
        "mech_richards": [
            ({"spline_s": 0.8}, 0.40),
            ({"spline_s": 0.5}, 0.40),
            ({"spline_s": 0.3}, 0.40),
        ],
        "mech_baranyi": [
            ({"spline_s": 0.8}, 0.40),
            ({"spline_s": 0.5}, 0.40),
            ({"spline_s": 0.5}, 0.40),
        ],
        "phenom_logistic": [
            ({"spline_s": 0.5}, 0.40),
            ({"spline_s": 0.3}, 0.40),
            ({"spline_s": 0.3}, 0.40),
        ],
        "phenom_gompertz": [
            ({"spline_s": 0.8}, 0.40),
            ({"spline_s": 0.5}, 0.40),
            ({"spline_s": 0.5}, 0.40),
        ],
        "phenom_gompertz_modified": [
            ({"spline_s": 0.05}, 0.40),
            ({"spline_s": 0.05}, 0.40),
            ({"spline_s": 0.05}, 0.40),
        ],
        "phenom_richards": [
            ({"spline_s": 0.8}, 0.40),
            ({"spline_s": 0.3}, 0.40),
            ({"spline_s": 0.3}, 0.40),
        ],
    },
}


def evaluate_nonparam_case(
    base_model_name, model_spec, case_idx, case, nonparam_method
):
    fit_kwargs, rel_tol = NONPARAM_CASE_CONFIG[nonparam_method][base_model_name][
        case_idx
    ]

    rel_errors = []
    fail_reasons = Counter()

    for seed in SEEDS:
        t_arr, N_noisy, true_params = model_spec["generate_func"](
            case["t"],
            noise_level=NOISE,
            seed=seed,
            **case["params"],
        )

        N_clean = evaluate_parametric_model(t_arr, base_model_name, true_params)
        true_mu_max = float(gc.inference.compute_mu_max(t_arr, N_clean))

        if not np.isfinite(true_mu_max) or true_mu_max <= 0:
            fail_reasons["invalid_true_mu"] += 1
            continue

        try:
            fit_result, stats = gc.fit_model(
                t_arr,
                N_noisy,
                model_name=nonparam_method,
                **fit_kwargs,
            )
        except Exception:
            fail_reasons["fit_exception"] += 1
            continue

        if fit_result is None or not isinstance(stats, dict):
            fail_reasons["fit_none"] += 1
            continue

        est_mu_max = stats.get("mu_max", np.nan)
        if not np.isfinite(est_mu_max) or est_mu_max <= 0:
            fail_reasons["invalid_estimated_mu"] += 1
            continue

        rel_err = abs(est_mu_max - true_mu_max) / abs(true_mu_max)
        rel_errors.append(rel_err)

    if len(rel_errors) == 0:
        raise AssertionError(
            f"[{nonparam_method} | {base_model_name} | {case['label']}] "
            "No valid mu_max estimates were produced. "
            f"Failure reasons: {dict(fail_reasons)}"
        )

    rel_errors = np.asarray(rel_errors, dtype=float)
    pass_rate = float(np.mean(rel_errors <= rel_tol))
    q50 = float(np.quantile(rel_errors, 0.50))
    q95 = float(np.quantile(rel_errors, 0.95))

    print(
        f"[{nonparam_method:14s}] {base_model_name:24s} {case['label'][:26]:26s} "
        f"pass_rate={pass_rate:.2%}  rel_tol={rel_tol:.2f}"
        f"  q50={q50:.3f}  q95={q95:.3f}"
    )

    assert pass_rate >= NONPARAM_PASS_RATE_MIN, (
        f"[{nonparam_method} | {base_model_name} | {case['label']}] "
        f"mu_max pass_rate={pass_rate:.2%} below {NONPARAM_PASS_RATE_MIN:.0%}. "
        f"q95={q95:.3f}, rel_tol={rel_tol:.2f}, fail_reasons={dict(fail_reasons)}"
    )


for nonparam_method in ["sliding_window", "spline"]:
    for base_model_name, model_spec in MODEL_SPECS.items():
        for case_idx, case in enumerate(model_spec["cases"]):
            evaluate_nonparam_case(
                base_model_name=base_model_name,
                model_spec=model_spec,
                case_idx=case_idx,
                case=case,
                nonparam_method=nonparam_method,
            )

print("\nAll non-parametric mu_max accuracy assertions passed.")

Running non-parametric mu_max accuracy tests...


[sliding_window] mech_logistic            short / low OD             pass_rate=100.00%  rel_tol=0.40  q50=0.058  q95=0.141


[sliding_window] mech_logistic            medium / mid OD            pass_rate=100.00%  rel_tol=0.40  q50=0.043  q95=0.083


[sliding_window] mech_logistic            long / high OD             pass_rate=100.00%  rel_tol=0.40  q50=0.057  q95=0.088


[sliding_window] mech_gompertz            short / low OD             pass_rate=100.00%  rel_tol=0.40  q50=0.130  q95=0.258


...

[sliding_window] phenom_gompertz_modified long / high OD             pass_rate=100.00%  rel_tol=0.40  q50=0.034  q95=0.035


[sliding_window] phenom_richards          short / nu=0.5             pass_rate=100.00%  rel_tol=0.40  q50=0.034  q95=0.068


[sliding_window] phenom_richards          medium / nu=1.0            pass_rate=100.00%  rel_tol=0.40  q50=0.015  q95=0.038


[sliding_window] phenom_richards          long / nu=2.0              pass_rate=100.00%  rel_tol=0.40  q50=0.019  q95=0.035
[spline        ] mech_logistic            short / low OD             pass_rate=100.00%  rel_tol=0.40  q50=0.087  q95=0.329
[spline        ] mech_logistic            medium / mid OD            pass_rate=32.00%  rel_tol=0.40  q50=0.455  q95=0.853


AssertionError: [spline | mech_logistic | medium / mid OD] mu_max pass_rate=32.00% below 95%. q95=0.853, rel_tol=0.40, fail_reasons={}

## Stress-Case Diagnostic: Baranyi High-Noise Failure Path
This is a diagnostic check only (does not fail the notebook).


In [ ]:
print("Running Baranyi stress-case diagnostic (historical high-noise failure seed)...")

t_reg = t_long
reg_params = {"mu": 0.25, "K": 3.00, "N0": 0.02, "h0": 1.5, "y0": 0.0}

t_bad, N_bad, _ = generate_mech_baranyi(
    t_reg,
    noise_level=0.006,
    seed=14,
    **reg_params,
)

try:
    reg_fit, reg_stats = gc.fit_model(t_bad, N_bad, model_name="mech_baranyi")
except Exception as exc:
    print("  Diagnostic result: fit raised an exception on this stress case.")
    print(f"  Exception type: {type(exc).__name__}")
    print(f"  Message: {exc}")
else:
    print("  Diagnostic result: no exception on this stress case.")
    print(f"  fit_result is None: {reg_fit is None}")
    print(f"  stats returned: {isinstance(reg_stats, dict)}")

## Data Validation and Failure-Mode Tests


In [ ]:
print("Running validate_data and invalid-input tests...")

# 1) validate_data removes invalid points and keeps finite positive values
t_dirty = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], dtype=float)
N_dirty = np.array(
    [0.0, 0.10, np.nan, 0.20, np.inf, -0.10, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80]
)

t_clean, N_clean = gc.inference.validate_data(t_dirty, N_dirty, min_points=5)
assert t_clean is not None and N_clean is not None, "Expected valid cleaned arrays"
assert np.all(np.isfinite(t_clean)) and np.all(
    np.isfinite(N_clean)
), "Expected finite cleaned arrays"
assert np.all(N_clean > 0), "Expected strictly positive cleaned OD values"

# 2) insufficient points returns (None, None)
t_short_bad = np.array([0, 1, 2, 3, 4], dtype=float)
N_short_bad = np.array([0.1, 0.2, 0.3, 0.4, 0.5], dtype=float)
vt, vN = gc.inference.validate_data(t_short_bad, N_short_bad, min_points=10)
assert vt is None and vN is None, "Expected None for insufficient number of points"

# 3) constant time axis returns (None, None)
t_const = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=float)
N_const = np.linspace(0.1, 1.0, 10)
vt2, vN2 = gc.inference.validate_data(t_const, N_const, min_points=10)
assert vt2 is None and vN2 is None, "Expected None for zero time variation"

# 4) fit_model on invalid data returns None + bad-fit stats
t_invalid = np.arange(8, dtype=float)
N_invalid = np.array([0.0, -1.0, np.nan, 0.0, 0.0, np.inf, -0.5, 0.1], dtype=float)
fit_invalid, stats_invalid = gc.fit_model(
    t_invalid, N_invalid, model_name="mech_logistic"
)

assert fit_invalid is None, "Expected fit_result=None on invalid input"
assert isinstance(stats_invalid, dict), "Expected bad-fit stats dict"
assert stats_invalid["fit_method"] is None, "Expected fit_method=None for bad fit"
assert np.isnan(stats_invalid["model_rmse"]), "Expected NaN RMSE for bad fit"

print("Validation and failure-mode tests passed.")

In [ ]:
print("\nAll tests passed.")
print(f"Seeds per case: {len(SEEDS)}")
print(f"Noise level: {NOISE}")
print(f"Minimum per-case pass rate: {PASS_RATE_MIN:.0%}")
print(f"Plots enabled: {SHOW_PLOTS}")